### RAG Application Using Type Sense

In [ ]:
import typesense

In [ ]:
client=typesense.Client({
  'nodes': [{
    'host': '',  # For Typesense Cloud use xxx.a1.typesense.net
    'port': '443',       # For Typesense Cloud use 443
    'protocol': 'https'    # For Typesense Cloud use https
  }],
  'api_key':'',
  'connection_timeout_seconds': 2
})



In [ ]:
books_schema = {
  'name': 'books',
  'fields': [
    {'name': 'title', 'type': 'string'},
    {'name': 'authors', 'type': 'string[]', 'facet': True},
    {'name': 'publication_year', 'type': 'int32', 'facet': True},
    {'name': 'ratings_count', 'type': 'int32'},
    {'name': 'average_rating', 'type': 'float'}
  ],
  'default_sorting_field': 'ratings_count'
}
print(client.collections.create(books_schema))

In [ ]:
search_parameters = {
  'q'         : 'harry potter',
  'query_by'  : 'title',
  'filter_by' : 'publication_year:<1998',
  'sort_by'   : 'publication_year:desc'
}

client.collections['books'].documents.search(search_parameters)

In [ ]:
search_parameters = {
  'q'         : 'experyment',
  'query_by'  : 'title',
  'facet_by'  : 'authors',
  'sort_by'   : 'average_rating:desc'
}

client.collections['books'].documents.search(search_parameters)

In [ ]:
### Langchain + Typsense + Groq LLM + RAG Application

from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Typesense
from langchain_text_splitters import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq

In [ ]:
import os
os.environ["GROQ_API_KEY"] = "" # set your Groq API key here

In [ ]:
loader = TextLoader("test.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

embeddings = HuggingFaceEmbeddings()

In [ ]:
docsearch=Typesense.from_documents(
    docs,
    embeddings,
    typesense_client_params={
        "host": "",  # Use xxx.a1.typesense.net for Typesense Cloud
        "port": "443",  # Use 443 for Typesense Cloud
        "protocol": "https",  # Use https for Typesense Cloud
        "typesense_api_key":"", # Your Typesense API Key
        "typesense_collection_name": "lang-chain"
    },
    
)

In [ ]:
query = "What is artificial intelligence"
found_docs = docsearch.similarity_search(query)
print(found_docs[0].page_content)

In [ ]:
### Retriever
retriever = docsearch.as_retriever()
retriever

In [ ]:
query = "Artificial intelligence indepth explanation"
retriever.invoke(query)[0]